In [1]:
import MDAnalysis as mda
from MDAnalysis.lib import distances
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import gaussian_kde
from scipy.interpolate import make_interp_spline, BSpline
# Calculate the center of mass for TRP rings
def compute_ring_coms(u, trp):
    coms = np.array([[u.atoms.select_atoms(f'(resname TRP and resid {id} and name C1 N1 C2 C3 C4 C5 C6 C7 C8)').center_of_mass()] for id in trp.residues.resids])
    return coms

def compute_distance(pos1, pos2, box_length):
    delta = pos1 - pos2
    delta -= np.round(delta / box_length) * box_length
    return np.linalg.norm(delta)

def compute_angle(vec1, vec2):
    cos_theta = np.dot(vec1, vec2)
    return np.degrees(np.arccos(np.clip(cos_theta, -1, 1)))

def compute_trp_normal(trp_res):
    C1 = trp_res.atoms.select_atoms("name C1").positions[0]
    C4 = trp_res.atoms.select_atoms("name C4").positions[0]
    C5 = trp_res.atoms.select_atoms("name C5").positions[0]
    vec1 = C4 - C1
    vec2 = C5 - C1
    normal = np.cross(vec1, vec2)
    norm = np.linalg.norm(normal)
    return normal / norm if norm != 0 else normal

def compute_ta_normal(ta_ring):
    points = ta_ring.positions[:6]
    vec1 = points[1] - points[4]
    vec2 = points[1] - points[3]
    normal = np.cross(vec1, vec2)
    norm = np.linalg.norm(normal)
    return normal / norm if norm != 0 else normal

def smooth(x, y):
    x_new = np.linspace(x.min(), x.max(), 300)
    y_smooth = make_interp_spline(x, y, k=2)
    return x_new, y_smooth(x_new)

edges = np.histogram([-1], 18, range=(0, 180))[1]
bins = 0.5 * (edges[:-1] + edges[1:])

parallel_angles = [(0, 30), (150, 180)]
herringbone_angles = [(30, 60), (120, 150)]
t_shaped_angles = [(60, 120)]

def check_stacking_type(angle):
    if 0 <= angle <= 30 or 150 <= angle <= 180:
        return 'parallel'
    elif 30 < angle < 60 or 120 <= angle < 150:
        return 'herringbone'
    elif 60 <= angle < 120:
        return 't_shaped'
    return None



In [ ]:
TPR = 'md_0_sim_sim_resp_24trp-1ta_500ns.tpr' 
XTC = 'md_0_sim_resp_24trp-1ta_500ns_pbc_center.xtc'
u = mda.Universe(TPR, XTC)
trp = u.select_atoms("resname TRP")

# Define TA aromatic rings
ta_rings = [
    u.select_atoms("name C38 C51 C63 C73 C62 C50"),
    u.select_atoms("name C93 C103 C111 C113 C112 C104"),
    u.select_atoms("name C25 C34 C45 C57 C46 C35"),
    u.select_atoms("name C79 C89 C10 C11 C101 C90"),
    u.select_atoms("name C18 C27 C36 C47 C37 C28"),
    u.select_atoms("name C70 C81 C91 C102 C92 C82"),
    u.select_atoms("name C15 C23 C32 C42 C33 C24"),
    u.select_atoms("name C66 C77 C87 C97 C88 C78"),
    u.select_atoms("name C21 C30 C40 C52 C41 C31"),
    u.select_atoms("name C75 C85 C95 C105 C96 C86")
]

In [ ]:
final_angles = []

stacking_counts = {'parallel': 0, 'herringbone': 0, 't_shaped': 0}

for ts in u.trajectory[5000:]:
    angles = []
    trp_coms = compute_ring_coms(u, trp)
    trp_normals = [compute_trp_normal(res) for res in trp.residues]
    ta_coms = [ring.center_of_mass() for ring in ta_rings]
    ta_normals = [compute_ta_normal(ring) for ring in ta_rings]
      
    for i, trp_com in enumerate(trp_coms):
        for j, ta_com in enumerate(ta_coms):
            dist = compute_distance(trp_com, ta_com, u.dimensions[:3])
            # print(dist)
            if dist <= 6.5:  
                angle = compute_angle(trp_normals[i], ta_normals[j])
                angles.append(angle)
                
    if len(angles) > 0:
        angle_hist = np.histogram(angles, 18, range=(0, 180), density=False)[0]*(100/len(angles))
        final_angles.append(angle_hist)

df_angle = pd.DataFrame(final_angles)
avg_final_angles = df_angle.mean(axis=0)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = [6.5, 5]

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams['axes.labelsize'] = 30
plt.rcParams["axes.linewidth"] = 2.5
plt.rcParams["axes.labelweight"] = 'bold'
plt.rcParams["lines.linewidth"] = 3

plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.top'] = True
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.right'] = True

plt.rcParams['xtick.major.size'] = 10
plt.rcParams['ytick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 3
plt.rcParams['ytick.major.width'] = 3

plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 3
plt.rcParams['ytick.minor.width'] = 3
plt.rcParams['xtick.minor.visible'] =  True
plt.rcParams['ytick.minor.visible'] =  True


plt.rcParams['legend.fontsize'] = 25
plt.rcParams['legend.frameon'] = False
plt.rcParams['legend.loc'] = 'best'
plt.rcParams['legend.markerscale'] = 1

plt.rcParams['savefig.dpi'] = 500
plt.rcParams['savefig.bbox'] = 'tight'

matplotlib.rc('font', weight='bold')

In [ ]:
fig, ax = plt.subplots()
# ax.pl
ax.plot(bins_new, avg_final_angles_smooth, color='red', lw=2)
ax.set_ylabel(r'$\mathbf{P(\theta)}$')
ax.set_xlabel(r'$\mathbf{\theta \; (degree)}$')
# ax.set_yticks([0, 0.004, 0.008, 0.012])
ax.set_xticks([0, 60, 120, 180])
# ax.axvspan(30, 60, alpha=0.2)
#ax.axvspan(60, 120, color='yellow', alpha=0.2)
# ax.axvspan(120, 150, alpha=0.2)

#ax.axvspan(0, 30, color='red', alpha=0.2)
#ax.axvspan(150, 180, color='red', alpha=0.2)
ax.xaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator(2))
ax.yaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator(2))
plt.xlim(0,180)
plt.savefig('Angle_distribution_TA-TRP.png', dpi=500,bbox_inches="tight")
plt.show()